In [11]:
from extract_puck_data import *

In [15]:
# try:
plt.rcParams['figure.figsize'] = [10, 10]
true_if_red = True
for traj in range(300, 1000):
    # try:
    path = f'/datastor1/calebc/public/data/mouse/cleaned_all/trajectory_data{traj}.hdf5'
    dataset_dict = {}
    dataset_dict = load_hdf5_to_dict(path)
    xs,ys = [], []
    # create_video_from_frames(dataset_dict['train_img'], f'traj_videos/traj_video{traj}.mp4')
    imgs = dataset_dict['train_img']#[150:160]
    for i, img in enumerate(imgs):
        train_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        dst, save_img = homography_transform(train_img)

        green_hsv = [[40,100,50], [80,255,255]]
        # red_hsv = [[0,100,50], [50,255,255]]
        red_hsv = [[0,100,50], [50,255,255]]
        

        if true_if_red:
            refined_img, idx,mask = find_hsv_puck(dst, color_hsv=red_hsv, apply_mask=True, glare=False )
        else:
            alternate_color = [[40,70,50], [80,255,255]]
            refined_img, idx,mask = find_hsv_puck(dst, color_hsv=green_hsv, apply_mask=True, color_alt=alternate_color)

        x,y = np.median(idx[0]), np.median(idx[1])

        
        MIN_PTS_CUTOFF = 500 if true_if_red else 200
        if len(idx[0]) < MIN_PTS_CUTOFF:
            xs.append(np.NaN)
            ys.append(np.NaN)
        else:
            xs.append(x)
            ys.append(y)
            
        if i == 0:
            img_puck_traj = dst 
            total_mask = mask
        else:
            total_mask = (total_mask | mask)
            img_puck_traj[:,:,0] = np.minimum(img_puck_traj[:,:,0], dst[:,:,0])
            img_puck_traj[:,:,1] = np.minimum(img_puck_traj[:,:,1], dst[:,:,1])
            img_puck_traj[:,:,2] = np.minimum(img_puck_traj[:,:,2], dst[:,:,2])
    
    xy_robot_frame = pixel2loc(xs, ys) #
    plt.figure()
    plt.subplot(2,2,1)
    plt.imshow(dst)
    plt.subplot(2,2,2)
    plt.plot(xy_robot_frame[1], xy_robot_frame[0])

    ax = plt.gca()  # Get current axes

    ax.set_aspect(aspect='equal', adjustable='box')
    # ax.invert_yaxis()
    
    plt.subplot(2,2,3)
    plt.imshow(img_puck_traj)
    plt.subplot(2,2,4)
    plt.imshow(total_mask)
    plt.savefig(f'../traj_videos/traj_plot{traj}.png')
    # plt.show()
    plt.clf()
    
    # Create a new HDF5 file
    save_dir = '../testt/'
    
    save_path = save_dir + f'state_trajectory_data{traj}.hdf5'
    dataset_dict['puck_state'] = xy_robot_frame
    dataset_dict['puck_state_nan_mask'] = np.isnan(xy_robot_frame)
    with h5py.File(save_path, 'w') as hdf5_file:
        print(dataset_dict.keys())
        save_dict_to_hdf5(dataset_dict, hdf5_file)
        
    print(f'finished traj {traj}')


KeyboardInterrupt: 